<a href="https://colab.research.google.com/github/biancailiescu/Digital-Menu/blob/main/insurance_taxonomy_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import spacy
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.cluster import HDBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Veridion challenge/insurance_taxonomy - insurance_taxonomy.csv'
df = pd.read_csv(file_path)

In [ ]:
nlp = spacy.load("en_core_web_sm")
def lemmatize(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""

    doc = nlp(text)
    filtered_tokens = []

    for token in doc:
        if token.pos_ in ["NOUN", "ADJ", "PROPN"] and not token.is_stop:
            filtered_tokens.append(token.lemma_.lower())

    return " ".join(filtered_tokens) if filtered_tokens else text.lower()

In [ ]:
df['lemmatized_name'] = df['label'].apply(lemmatize)

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = model.encode(df['lemmatized_name'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
umap_emb = UMAP(n_components=5, random_state=42).fit_transform(embeddings)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [ ]:
from sklearn.cluster import KMeans
num_clusters = 40
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)

In [ ]:
df['cluster'] = clusters

In [ ]:
clusters = df.groupby('cluster')

for cluster_id, cluster_data in clusters:
    print(f"Cluster {cluster_id}:")
    print(cluster_data[['label']])
    print("\n")

Cluster 0:
                      label
81        Painting Services
82  Spray Painting Services


Cluster 1:
                         label
33  Street Cleaning Operations
99  Building Cleaning Services


Cluster 2:
                             label
75                Drywall Services
133  Carpet Manufacturing Services
140    Tent Manufacturing Services


Cluster 3:
                             label
185              Training Services
200    Corporate Training Services
208  Community Engagement Services
209           Stakeholder Services
216               Fitness Coaching
218      Physical Therapy Services


Cluster 4:
                                      label
23            Fencing Construction Services
48           Pipeline Construction Services
73                Tilt-Up Concrete Services
74            Masonry Construction Services
77                       Carpentry Services
84                 Interior Design Services
95             Residential Roofing Services
102  Single Family Resi

In [ ]:
condition = df["label"].str.contains("Software", na=False)

df.loc[condition, "cluster"] = 40

print(df["cluster"].value_counts())


cluster
11    17
4     12
13    11
32    10
16    10
5      9
12     8
28     7
8      7
22     7
21     6
3      6
25     6
19     6
20     6
33     6
24     5
30     5
37     5
6      5
39     5
27     5
18     5
14     4
9      4
29     4
23     4
26     4
2      3
17     3
10     3
31     3
38     3
36     3
35     2
7      2
1      2
15     2
0      2
40     2
34     1
Name: count, dtype: int64


In [ ]:
silhouette_avg = silhouette_score(embeddings, df['cluster'])
davies_bouldin_avg = davies_bouldin_score(embeddings, df['cluster'])

print(f"Silhouette Score: {silhouette_avg:.2f}")
print(f"Davies-Bouldin Score: {davies_bouldin_avg:.2f}")

Silhouette Score: 0.07
Davies-Bouldin Score: 1.98


In [ ]:
insurance_category = {
    0: "Painting Services",
    1: "Cleaning Operations",
    2: "Manufacturing Services",
    3: "Training Services",
    4: "Construction Services",
    5: "House Services",
    6: "Animal Food Processing",
    7: "Signage Installation",
    8: "Gas Installation",
    9: "Furniture Manufacturing",
    10: "Communication Equipment",
    11: "Consulting Services",
    12: "Veterinary Services",
    13: "Plumbing Services",
    14: "Snow Removal Services",
    15: "Seeds",
    16: "Property Management Services",
    17: "Window Manufacturing",
    18: "Social Media Services",
    19: "Accessory Manufacturing",
    20: "Volunteering Services",
    21: "Data Analysis Services",
    22: "Excavation",
    23: "Media Production Services",
    24: "Agricultural Services",
    25: "Road Construction",
    26: "Marketing Services",
    27: "Boiler Installation",
    28: "Plastic Manufacturing",
    29: "Foundation Construction",
    30: "Apparel Manufacturing",
    31: "Drainage",
    32: "Food Processing",
    33: "Fire Protection",
    34: "Tree Services",
    35: "Soil",
    36: "Glass Installation",
    37: "Paper Services",
    38: "Electrical Services",
    39: "Health Services",
    40: "Software Development"
    }


df['insurance_category'] = df['cluster'].map(insurance_category)

In [ ]:
df.drop(columns=['lemmatized_name'], inplace=True)

In [ ]:
print(df['insurance_category'])

0             Agricultural Services
1                              Soil
2                              Soil
3           Accessory Manufacturing
4      Property Management Services
                   ...             
215             Consulting Services
216               Training Services
217                 Health Services
218               Training Services
219                 Health Services
Name: insurance_category, Length: 220, dtype: object


In [ ]:
output_file_path = '/content/drive/MyDrive/Colab Notebooks/Veridion challenge/labels.csv'
df.to_csv(output_file_path, index=False)
print(f"File saved to {output_file_path}")

File saved to /content/drive/MyDrive/Colab Notebooks/Veridion challenge/labels.csv
